In [41]:
# importer les modules 
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
import lxml.html as lh
import pandas as pd
import numpy as np
import re
import json
from bs4 import BeautifulSoup

In [42]:
# tokeniseur pré-entraîné sur plusieurs langues
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
# modèle de classification de séquences en plusieurs langues
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [69]:
# URL Allociné du film Inception
url = 'https://www.allocine.fr/film/fichefilm-143692/critiques/spectateurs/'
# Chaîne à ajouter à l'URL pour la pagination
uri_pages = '?page='
# Nombre de pages à récupérer 
nb_pages = 135
# Liste des chemins XPath 
chemins = ['//span[@class="stareval-note"]', \ # note de l'avis
'//div[@class="content-txt review-card-content"]' ] # texte de l'avis



# Liste des noms de colonnes pour les données extraites
colonnes = ['Note', 'Description']


def recuperer_page(url): # récupérer le contenu d'une pages 
  page = requests.get(url)
  page.encoding = 'utf-8'
  soup = BeautifulSoup(page.content, 'lxml')

  content = []
  # Extraction des données en utilisant les méthodes de BeautifulSoup
  for tag in chemins:
    elements = soup.select(tag)
    content.append([element.text.strip() for element in elements]) 

  # Création d'une liste de dictionnaires à partir des données extraites
  avis = []
  for i in range(len(content[0])):
    # Création d'un dictionnaire pour chaque avis
    review = dict(zip(colonnes, [content[j][i] for j in range(len(content))]))
    avis.append(review)
  return avis


def recuperer_toutes_pages(nb_pages, url): # récupérer le contenu de toutes les pages
  critiques = []
  for i in range(nb_pages):
    url_page = url + uri_pages + str(i + 1)
    critiques.extend(recuperer_page(url_page))
  return critiques # renvoie la liste des pages récupérées 


# Récupération de toutes les critiques
avis = recuperer_toutes_pages(nb_pages, url)

# Enregistrement des critiques dans un fichier JSON
with open('avis.json', 'w', encoding='utf-8') as fichier:
  # `ensure_ascii=False` permet de conserver les caractères non-ASCII
  json.dump(avis, fichier, ensure_ascii=False)

In [30]:
# lire le fichier JSON contenant les critiques
with open("avis.json", 'r') as f:
  json_data = json.load(f)

# Convertir les données JSON en DataFrame
df = pd.DataFrame(json_data)

# afficher le DataFrame
print(df)

                                            Description
0     Après avoir vu ce fameux Oppenheimer en 70mm :...
1     3,25/5. J'ai du mal à comprendre certains supe...
2     Avec Oppenheimer, Christopher Nolan nous proje...
3     Beaucoup, beaucoup, beaucoup de dialogues.Beau...
4     Fait rarissime, j'ai quitté la salle au bout d...
...                                                 ...
3867  Film largement surcoté tout comme ce realisate...
3868  Incroyable du grand Nolan.Il arrive toujours à...
3869  Je trouve le film sauvé par la prestation de M...
3870  Film un peu long, surtout à la fin de la premi...
3871  C'est une claque. Le film est long, il peut né...

[3872 rows x 1 columns]


In [31]:
# convertir le DataFrame en un tableau array
df = pd.DataFrame(np.array(df), columns=['Description']) 

In [35]:
# séléctionner la première ligne de la colonne Description
df['Description'].iloc[1]  

'3,25/5. J\'ai du mal à comprendre certains superlatifs à propos de ce film et, à ce titre, je m\'attendais à quand même mieux. Pourtant, je ne suis pas surpris : c\'est un biopic, et comme souvent dans les biopics, on tourne beaucoup autour des relations individuelles de l\'intéressé, ce qui se traduit par d\'innombrables scènes de dialogues. Ce film ne fait pas exception, au moins 2h30 des 2h50 du film nous montrent des séquences de dialogues entre individus en milieux clos. Dans des salles diverses, lieux d\'études, bureaux, au parlement, entre deux arbres, en voiture... Il y a bien quelques brefs plans de l\'immensité du désert pour planter le lieu de l\'opération Manhattan, et une minute d\'explosion nucléaire. Insuffisant à mon goût pour trois heures que j\'ai trouvées un petit peu longues dans l\'absolu. Autre conséquence également : n\'allez pas voir ce film en IMAX ou je ne sais quelle salle fabuleusement extraordinaire. Il n\'y a quasiment aucun intérêt au regard des images q

In [36]:
def sentiment_score(avis):
    # tokeniser, vectoriser et renvoyer un tensor PyTorch
    tokens = tokenizer.encode(avis, return_tensors='pt')
    resultat = model(tokens) # appliquer le modèle aux avis 
    return int(torch.argmax(resultat.logits))+1 # renvoyer un nombre entier entre 1 et 5 

In [37]:
# appliquer la fonction sentiment_score sur le premier avis
sentiment_score(df['Description'].iloc[0]) 

3

In [38]:
# créer une nouvelle colonnes 'sentiment' qui contient le sentiment_score de chaque avis
df['sentiment'] = df['Description'].apply(lambda x: sentiment_score(x[:512])) 

In [27]:
df

,Description,sentiment
0,Il serait difficile de mettre moins de 3 étoil...,3
1,impossible de mettre une mauvaise note car vis...,2
2,Enfin de retour sur Pandora treize ans après l...,4
3,"Qualité constante chez Cameron, celle de dépas...",4
4,On nous a promit que les leçons du 1er film on...,2
...,...,...
5995,"Ce film est magnifique, bien qu’il soit long n...",5
5996,A VOIR POUR LE PLAISIR DES YEUXJe ne regrette ...,3
5997,"Pour ma première critique, ce sera donc Avatar...",1
5998,Mise à part qualité de l’image le film est pla...,2
